In [ ]:
import os
import json
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate
from utils import read_notebook, format_for_gpt, extract_cells, create_prompt_example

In [ ]:
example_1,human_feedback_example_1 = create_prompt_example(path_to_example="",human_grade="",human_feedback="")
example_2,human_feedback_example_2 = create_prompt_example(path_to_example="",human_grade="",human_feedback="")
example_3,human_feedback_example_3 = create_prompt_example(path_to_example="",human_grade="",human_feedback="")


examples = [
    {"input": example_1, "output": human_feedback_example_1},
    {"input": example_2, "output": human_feedback_example_2},
    {'input': example_3, "output": human_feedback_example_3}
]

In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    
    input_variables=["input"],
    examples=examples,
    example_prompt= example_prompt
    )

In [ ]:
OpenAI_api_key = ""

if OpenAI_api_key == "":
    print("please provide your openAI api key")
chat_model = ChatOpenAI(temperature=0, model_name="gpt-4", openai_api_key=OpenAI_api_key)

In [ ]:

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that evaluate programming assignments and check all the instructions are followed and grade it from 1-100 and provide Feedback what went wrong.Provide Reasoning on why you gave such grade. \
            Given the dataset about weather in Australia , predict rain using models. \
         - must have used ML model to perform prediction. \
          - Include VectorAssembler successfully to perform feature transformation  "
),
        
        few_shot_prompt,
        ("user", "{input}"),
    ]
)
chain = final_prompt | chat_model


In [ ]:
folder_path = 'path to inference notebook folder'
file_names,files = [],[]
# Collecting .ipynb files and avoiding hidden/system files
for file in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file)
    if os.path.isfile(full_path) and file.endswith('.ipynb') and not file.startswith('.'):
        file_names.append(full_path)
        



In [ ]:
#### To avoid RateLimitException taking 3 assignments per loop
output = []
for file in file_names[:]:
    notebook = read_notebook(file)
    extracted_cells = extract_cells(notebook)
    example = format_for_gpt(extracted_cells)
    response_with_context = chain.invoke({"input": example})
    output.append(response_with_context)


In [ ]:

with open('gpt_4_inference.jsonl', 'w') as file:
    for i,evaluation in enumerate(output):
        json_line = json.dumps(evaluation.content)  # Converting dictionary to JSON string
        file.write(json_line + '\n') 

In [ ]:
for i,evaluation in enumerate(output):
    feedback = evaluation.content.split("\n")
    print("printing feedback for {}".format(file_names[i]))
    for points in feedback:
        print(points)